In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [19]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-02-27 16:28:22,688 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:28:22,711 INFO: Initializing external client
2025-02-27 16:28:22,712 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:28:23,699 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648
Fetching data from 2025-01-29 21:28:22.683438+00:00 to 2025-02-27 20:28:22.683438+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.34s) 


In [20]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-27 16:28:32,329 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:28:32,332 INFO: Initializing external client
2025-02-27 16:28:32,332 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:28:33,061 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648

In [21]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [22]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,84,0.0,2025-02-27 22:00:00+00:00
1,189,0.0,2025-02-27 22:00:00+00:00
2,197,1.0,2025-02-27 22:00:00+00:00
3,156,0.0,2025-02-27 22:00:00+00:00
4,247,0.0,2025-02-27 22:00:00+00:00
...,...,...,...
253,58,0.0,2025-02-27 22:00:00+00:00
254,183,0.0,2025-02-27 22:00:00+00:00
255,79,101.0,2025-02-27 22:00:00+00:00
256,132,122.0,2025-02-27 22:00:00+00:00


In [23]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-02-27 16:28:43,883 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-27 16:28:43,887 INFO: Initializing external client
2025-02-27 16:28:43,887 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-27 16:28:44,667 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648


In [24]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |███████████████████████████████████████| Rows 258/258 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214648/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)